In [1]:
import pandas as pd
import numpy as np
import sqlite3
import statsmodels.formula.api as smf

from regtabletotext import prettify_result
from statsmodels.regression.rolling import RollingOLS
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import percent_format, date_format
from joblib import Parallel, delayed, cpu_count
from itertools import product


tidy_finance = sqlite3.connect(database="data/tidy_finance_python.sqlite")

crsp_monthly = (pd.read_sql_query(
    sql="SELECT permno, month, industry, ret_excess FROM crsp_monthly",
    con=tidy_finance,
    parse_dates={"month"})
  .dropna()
)

factors_ff3_monthly = pd.read_sql_query(
  sql="SELECT month, mkt_excess FROM factors_ff3_monthly",
  con=tidy_finance,
  parse_dates={"month"}
)

crsp_monthly = (crsp_monthly
  .merge(factors_ff3_monthly, how="left", on="month")
)

In [2]:
tidy_finance = sqlite3.connect(database="data/tidy_finance_python.sqlite")

crsp_monthly = (pd.read_sql_query(
    sql="SELECT permno, month, industry, ret_excess FROM crsp_monthly",
    con=tidy_finance,
    parse_dates={"month"})
  .dropna()
)

factors_ff3_monthly = pd.read_sql_query(
  sql="SELECT month, mkt_excess FROM factors_ff3_monthly",
  con=tidy_finance,
  parse_dates={"month"}
)

crsp_monthly = (crsp_monthly
  .merge(factors_ff3_monthly, how="left", on="month")
)

model_beta = (smf.ols(
    formula="ret_excess ~ mkt_excess",
    data=crsp_monthly.query("permno == 14593"))
  .fit()
)
prettify_result(model_beta)